In [ ]:
import os
import pickle
import numpy as np
import pandas as pd
import pymc3 as pm3
import seaborn as sns
import matplotlib.pylab as plt

%matplotlib inline

In [ ]:
data = pd.read_csv('../testresults/simdata2D.csv', sep=',')
print(data.shape)
data.head()

In [ ]:
X = data[['x1', 'x2']].T.values
X_gp = data[['centroid_x', 'centroid_y']].values
y = data.log_crime_rate.values
num_features = X.shape[0]  # there are 2 features (x1,x2)
X_mean = X.mean(axis=1, keepdims=True)
X_centered = X-X_mean # required to speed up the process; without scaling it takes > 1hour fir 11000 samples but with scaling it becomes ~15min
print(X.shape, X_mean.shape, X_centered.shape, X_gp.shape, y.shape)
#Note; y.shape must not be (400,1) but it must be (400,). The same X must be (2, 400) not (400,2); otherwise, it takes too much time with wrong result

In [ ]:
gp_input_dim = 2
time_involved = False

if os.path.isfile('../chain.pickle'):
    chain = pickle.load(open('../chain.pickle', 'rb'))
else:
    with pm3.Model() as model_mlr:
        ## Bayesian linear regression
        α_blr = pm3.Normal('α_blr', mu=0, sd=10)
        β_blr = pm3.Normal('β_blr', mu=0, sd=1, shape=num_features)
        σ = pm3.HalfCauchy('σ', 5)
        μ_blr = α_blr + pm3.math.dot(β_blr, X_centered)

        ## The spatial GP
        η_spatial_trend = 1 #pm3.HalfCauchy("η_trend", beta=2, testval=2.0)
        ℓ_spatial_trend = pm3.Gamma("ℓ_trend", alpha=4, beta=0.1, shape=gp_input_dim)
        cov_spatial_trend = (
            η_spatial_trend**2 * pm3.gp.cov.ExpQuad(input_dim=gp_input_dim, ls=ℓ_spatial_trend)
        )
        gp_spatial_trend = pm3.gp.Marginal(cov_func=cov_spatial_trend)    

        gp = gp_spatial_trend
        if time_involved:
            ## The temporal GP    
            # yearly periodic component
            yearly_period  = pm3.Normal("yearly_period", mu=1, sd=0.05)
            yearly_smooth = pm3.Gamma("yearly_smooth ", alpha=4, beta=3)
            cov_yearly = pm3.gp.cov.Periodic(1, yearly_period, yearly_smooth)
            gp_yearly = pm3.gp.Marginal(cov_func=cov_yearly)

            # weekly periodic component
            weekly_period  = pm3.Normal("weekly_period", mu=1, sd=0.05)
            weekly_smooth = pm3.Gamma("weekly_smooth ", alpha=4, beta=3)
            cov_weekly = pm3.gp.cov.Periodic(1, weekly_period, weekly_smooth)
            gp_weekly = pm3.gp.Marginal(cov_func=cov_weekly)

            gp += gp_yearly + gp_weekly

        # noise model
        cov_noise = pm3.gp.cov.WhiteNoise(σ)
        
        # alpha_tmp is for centerlized data. It must be trnasformed to original.
        y_blr = pm3.Normal(
            'y_blr', 
            mu=μ_blr, 
            sd=σ, 
            observed=y)
        y_gp = gp.marginal_likelihood("y_gp", X=X_gp, y=y-μ_blr, noise=cov_noise)

        start = None # pm3.find_MAP()
        step = pm3.Metropolis() # pm3.NUTS(scaling=start)
        trace_mlr = pm3.sample(5000, start=start, step=step)
        chain = trace_mlr[-5000:]
        pickle.dump(chain, open('../chain.pickle', 'wb'))

varnames = ['α_blr', 'β_blr', 'σ', 'ℓ_trend'] #'η_trend', 
pm3.traceplot(chain, varnames=varnames)
pm3.autocorrplot(chain)
print(pm3.summary(chain, varnames))